<a href="https://colab.research.google.com/github/skyMei-J/Campus-Dog/blob/main/DecisionTree_RandomForest_Gaussian_Method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import numpy as np
from torchvision import datasets, transforms
from torch.utils.checkpoint import checkpoint


# decision tree

In [ ]:
from __future__ import print_function

from gluoncv import model_zoo, data, utils
from matplotlib import pyplot as plt

from sklearn.svm import SVC
from skimage import exposure
from skimage import feature
from imutils import paths
from sklearn import tree
from sklearn import ensemble
import os
import glob

import argparse
import imutils
import cv2

import numpy as np
import os
import glob
import random
import pandas as pd
import time

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

net = model_zoo.get_model('yolo3_darknet53_voc', pretrained=True)
def cutImage(imageP):
    print(imageP)
    x, img = data.transforms.presets.yolo.load_test(imageP, short=512)

    class_IDs, scores, bounding_boxs = net(x)
    ax = utils.viz.plot_bbox(img, bounding_boxs[0], scores[0],class_IDs[0], class_names=net.classes)
    bbbox = bounding_boxs[0][0].astype(int).asnumpy()

    (x,y,w,h) = bbbox
    if x<0 :
        x=0
    if y<0:
        y=0
    img = img[y:y+h,x:x+w]
    plt.imshow(img)
    plt.show()
    return img


#####################################################

labels = []
count = int(0)
data = np.empty((0, 6),float)
cutted_data = np.empty((0, 6),float)
entries = os.listdir("train")
output = pd.DataFrame({'Mean_Blue': [], 'Mean_Green': [], 'Mean_Red': [], 'Standard_Blue': [], 'Standard_Green': [], 'Standard_Red': [], 'Label': []})
cutted_output = pd.DataFrame({'Mean_Blue': [], 'Mean_Green': [], 'Mean_Red': [], 'Standard_Blue': [], 'Standard_Green': [], 'Standard_Red': [], 'Label': []})
allimage_list = []
total_data_num = 0
for dirPath in entries:
    allimage = glob.glob( "train/"+dirPath +"/*.jpg")
    for i in allimage:
        allimage_list.append(i)

for dirPath in entries:
    allimage = glob.glob( "train/"+dirPath +"/*.JPG")
    for i in allimage:
        allimage_list.append(i)

total_data_num = len(allimage_list)
temp_data_num = total_data_num
# print("total_data_num: ", total_data_num)

while temp_data_num > total_data_num * 0.3:

    #read path
    imagePath = allimage_list.pop(random.randint(0, temp_data_num-1))
    temp_data_num = temp_data_num - 1
    # print("temp_data_num: ", temp_data_num)
    #make = label
    make = imagePath.split("/")[-2]

    ########## original ##########
    image = cv2.imread(imagePath)
    # print(imagePath)
    (means, stds) = cv2.meanStdDev(cv2.cvtColor(image, cv2.COLOR_BGR2HSV))
    result = pd.DataFrame({'Mean_Blue': [means[0]], 'Mean_Green': [means[1]], 'Mean_Red': [means[2]], 'Standard_Blue': [stds[0]], 'Standard_Green': [stds[1]], 'Standard_Red': [stds[2]], 'Label': [make]})
    output = pd.concat([output, result], axis = 0)

    x_data = []
    for i in means:
        x_data.append(float(i))
    for i in stds:
        x_data.append(float(i))
    x_data = np.array(x_data)

    data = np.vstack((data, x_data))


    ########## cutted ##########
    # print(imagePath,'############################')
    # plt.imshow(image)
    # plt.show()
    # cutted_image = cutImage(imagePath)
    # (means, stds) = cv2.meanStdDev(cv2.cvtColor(cutted_image, cv2.COLOR_BGR2HSV))
    # cutted_result = pd.DataFrame({'Mean_Blue': [means[0]], 'Mean_Green': [means[1]], 'Mean_Red': [means[2]], 'Standard_Blue': [stds[0]], 'Standard_Green': [stds[1]], 'Standard_Red': [stds[2]], 'Label': [make]})
    # cutted_output = pd.concat([output, result], axis = 0)

    # cutted_x_data = []
    # for i in means:
        # cutted_x_data.append(float(i))
    # for i in stds:
        # cutted_x_data.append(float(i))
    # cutted_x_data = np.array(cutted_x_data)

    # cutted_data = np.vstack((cutted_data, cutted_x_data))

    #push label into label list
    labels.append(make)

data = np.array(data)
cutted_data = np.array(cutted_data)

clf = ensemble.RandomForestClassifier(n_estimators = 99)
clf = clf.fit(data, labels)

# cutted_clf = ensemble.RandomForestClassifier(n_estimators = 99)
# cutted_clf = cutted_clf.fit(cutted_data, labels)

correct = 0
cutted_correct = 0
num_test_data = 0

tstart = time.time()

while temp_data_num:

    temp_data_num = temp_data_num - 1
    # print("temp_data_num: ", temp_data_num)
    imagePath = allimage_list.pop()
    make = imagePath.split("/")[-2]
    num_test_data = num_test_data + 1

    ########## original ##########
    image = cv2.imread(imagePath)
    (means, stds) = cv2.meanStdDev(cv2.cvtColor(image, cv2.COLOR_BGR2HSV))
    result = pd.DataFrame({'Mean_Blue': [means[0]], 'Mean_Green': [means[1]], 'Mean_Red': [means[2]], 'Standard_Blue': [stds[0]], 'Standard_Green': [stds[1]], 'Standard_Red': [stds[2]], 'Label': [make]})
    output = pd.concat([output, result], axis = 0)

    x_data = []
    for i in means:
        x_data.append(float(i))
    for i in stds:
        x_data.append(float(i))
    x_data = np.array(x_data)
    x_data = x_data.reshape(1,-1)
    pred = clf.predict(x_data)

    ########## cutted ##########
    # cutted_image = cutImage(imagePath)
    # (means, stds) = cv2.meanStdDev(cv2.cvtColor(cutted_image, cv2.COLOR_BGR2HSV))
    # cutted_result = pd.DataFrame({'Mean_Blue': [means[0]], 'Mean_Green': [means[1]], 'Mean_Red': [means[2]], 'Standard_Blue': [stds[0]], 'Standard_Green': [stds[1]], 'Standard_Red': [stds[2]], 'Label': [make]})
    # cutted_output = pd.concat([output, result], axis = 0)

    # cutted_x_data = []
    # for i in means:
        # cutted_x_data.append(float(i))
    # for i in stds:
        # cutted_x_data.append(float(i))
    # cutted_x_data = np.array(cutted_x_data)
    # cutted_x_data = x_data.reshape(1,-1)
    # cutted_pred = cutted_clf.predict(cutted_x_data)


    #original result	
    # print("original")
    # print("target: ", make)
    # print("predict: ", pred)

    if make == pred:
        correct = correct + 1

    #cutted result	
    # print("cutted")
    # print("target: ", make)
    # print("predict: ", cutted_pred)

    # if make == cutted_pred:
        # cutted_correct = cutted_correct + 1

print("Original")
print(correct/num_test_data)
# print("Cutted")
# print(cutted_correct/num_test_data)

tend = time.time()

print("time : ", total_data_num/(tend - tstart))

output.to_csv('Output.csv')
fig1 = plt.figure()
ax1 = Axes3D(fig1)
ax1.set_xlim3d(output['Mean_Blue'].min(), output['Mean_Blue'].max())
ax1.set_ylim3d(output['Mean_Green'].min(), output['Mean_Green'].max())
ax1.set_zlim3d(output['Mean_Red'].min(), output['Mean_Red'].max())
for i in range(output.shape[0]):
    if output.iloc[i]['Label'] == 'white':
        ax1.plot(output.iloc[i]['Mean_Blue'], output.iloc[i]['Mean_Green'], output.iloc[i]['Mean_Red'], 'ro')
    elif output.iloc[i]['Label'] == 'steak':
        ax1.plot(output.iloc[i]['Mean_Blue'], output.iloc[i]['Mean_Green'], output.iloc[i]['Mean_Red'], 'go')
    elif output.iloc[i]['Label'] == 'curl':
        ax1.plot(output.iloc[i]['Mean_Blue'], output.iloc[i]['Mean_Green'], output.iloc[i]['Mean_Red'], 'bo')
    elif output.iloc[i]['Label'] == 'dressed_long_leg':
        ax1.plot(output.iloc[i]['Mean_Blue'], output.iloc[i]['Mean_Green'], output.iloc[i]['Mean_Red'], 'co')
    elif output.iloc[i]['Label'] == 'long leg':
        ax1.plot(output.iloc[i]['Mean_Blue'], output.iloc[i]['Mean_Green'], output.iloc[i]['Mean_Red'], 'mo')
    else:
        ax1.plot(output.iloc[i]['Mean_Blue'], output.iloc[i]['Mean_Green'], output.iloc[i]['Mean_Red'], 'ko')
plt.show()

fig2 = plt.figure()
ax2 = Axes3D(fig2)
ax2.set_xlim3d(output['Standard_Blue'].min(), output['Standard_Blue'].max())
ax2.set_ylim3d(output['Standard_Green'].min(), output['Standard_Green'].max())
ax2.set_zlim3d(output['Standard_Red'].min(), output['Standard_Red'].max())
for i in range(output.shape[0]):
    if output.iloc[i]['Label'] == 'white':
        ax2.plot(output.iloc[i]['Standard_Blue'], output.iloc[i]['Standard_Green'], output.iloc[i]['Standard_Red'], 'ro')
    elif output.iloc[i]['Label'] == 'steak':
        ax2.plot(output.iloc[i]['Standard_Blue'], output.iloc[i]['Standard_Green'], output.iloc[i]['Standard_Red'], 'go')
    elif output.iloc[i]['Label'] == 'curl':
        ax2.plot(output.iloc[i]['Standard_Blue'], output.iloc[i]['Standard_Green'], output.iloc[i]['Standard_Red'], 'bo')
    elif output.iloc[i]['Label'] == 'dressed_long_leg':
        ax2.plot(output.iloc[i]['Standard_Blue'], output.iloc[i]['Standard_Green'], output.iloc[i]['Standard_Red'], 'co')
    elif output.iloc[i]['Label'] == 'long leg':
        ax2.plot(output.iloc[i]['Standard_Blue'], output.iloc[i]['Standard_Green'], output.iloc[i]['Standard_Red'], 'mo')
    else:
        ax2.plot(output.iloc[i]['Standard_Blue'], output.iloc[i]['Standard_Green'], output.iloc[i]['Standard_Red'], 'ko')
plt.show()

# cutted_output.to_csv('Cutted_Output.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'train'

# random forest

In [ ]:
from __future__ import print_function

from gluoncv import model_zoo, data, utils
from matplotlib import pyplot as plt

from sklearn.svm import SVC
from skimage import exposure
from skimage import feature
from imutils import paths
from sklearn import tree
from sklearn import ensemble
import os
import glob

import argparse
import imutils
import cv2

import numpy as np
import os
import glob
import random
import pandas as pd
import time

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

net = model_zoo.get_model('yolo3_darknet53_voc', pretrained=True)
def cutImage(imageP):
    print(imageP)
    x, img = data.transforms.presets.yolo.load_test(imageP, short=512)

    class_IDs, scores, bounding_boxs = net(x)
    ax = utils.viz.plot_bbox(img, bounding_boxs[0], scores[0],class_IDs[0], class_names=net.classes)
    bbbox = bounding_boxs[0][0].astype(int).asnumpy()

    (x,y,w,h) = bbbox
    if x<0 :
        x=0
    if y<0:
        y=0
    img = img[y:y+h,x:x+w]
    plt.imshow(img)
    plt.show()
    return img


#####################################################

labels = []
count = int(0)
data = np.empty((0, 6),float)
cutted_data = np.empty((0, 6),float)
entries = os.listdir("train")
output = pd.DataFrame({'Mean_Blue': [], 'Mean_Green': [], 'Mean_Red': [], 'Standard_Blue': [], 'Standard_Green': [], 'Standard_Red': [], 'Label': []})
cutted_output = pd.DataFrame({'Mean_Blue': [], 'Mean_Green': [], 'Mean_Red': [], 'Standard_Blue': [], 'Standard_Green': [], 'Standard_Red': [], 'Label': []})
allimage_list = []
total_data_num = 0
for dirPath in entries:
    allimage = glob.glob( "train/"+dirPath +"/*.jpg")
    for i in allimage:
        allimage_list.append(i)

for dirPath in entries:
    allimage = glob.glob( "train/"+dirPath +"/*.JPG")
    for i in allimage:
        allimage_list.append(i)

total_data_num = len(allimage_list)
temp_data_num = total_data_num
# print("total_data_num: ", total_data_num)

while temp_data_num > total_data_num * 0.3:

    #read path
    imagePath = allimage_list.pop(random.randint(0, temp_data_num-1))
    temp_data_num = temp_data_num - 1
    # print("temp_data_num: ", temp_data_num)
    #make = label
    make = imagePath.split("/")[-2]

    ########## original ##########
    image = cv2.imread(imagePath)
    # print(imagePath)
    (means, stds) = cv2.meanStdDev(cv2.cvtColor(image, cv2.COLOR_BGR2HSV))
    result = pd.DataFrame({'Mean_Blue': [means[0]], 'Mean_Green': [means[1]], 'Mean_Red': [means[2]], 'Standard_Blue': [stds[0]], 'Standard_Green': [stds[1]], 'Standard_Red': [stds[2]], 'Label': [make]})
    output = pd.concat([output, result], axis = 0)

    x_data = []
    for i in means:
        x_data.append(float(i))
    for i in stds:
        x_data.append(float(i))
    x_data = np.array(x_data)

    data = np.vstack((data, x_data))


    ########## cutted ##########
    # print(imagePath,'############################')
    # plt.imshow(image)
    # plt.show()
    # cutted_image = cutImage(imagePath)
    # (means, stds) = cv2.meanStdDev(cv2.cvtColor(cutted_image, cv2.COLOR_BGR2HSV))
    # cutted_result = pd.DataFrame({'Mean_Blue': [means[0]], 'Mean_Green': [means[1]], 'Mean_Red': [means[2]], 'Standard_Blue': [stds[0]], 'Standard_Green': [stds[1]], 'Standard_Red': [stds[2]], 'Label': [make]})
    # cutted_output = pd.concat([output, result], axis = 0)

    # cutted_x_data = []
    # for i in means:
        # cutted_x_data.append(float(i))
    # for i in stds:
        # cutted_x_data.append(float(i))
    # cutted_x_data = np.array(cutted_x_data)

    # cutted_data = np.vstack((cutted_data, cutted_x_data))

    #push label into label list
    labels.append(make)

data = np.array(data)
cutted_data = np.array(cutted_data)

clf = ensemble.DecisionTreeClassifier(n_estimators = 99)
clf = clf.fit(data, labels)

# cutted_clf = ensemble.RandomForestClassifier(n_estimators = 99)
# cutted_clf = cutted_clf.fit(cutted_data, labels)

correct = 0
cutted_correct = 0
num_test_data = 0

tstart = time.time()

while temp_data_num:

    temp_data_num = temp_data_num - 1
    # print("temp_data_num: ", temp_data_num)
    imagePath = allimage_list.pop()
    make = imagePath.split("/")[-2]
    num_test_data = num_test_data + 1

    ########## original ##########
    image = cv2.imread(imagePath)
    (means, stds) = cv2.meanStdDev(cv2.cvtColor(image, cv2.COLOR_BGR2HSV))
    result = pd.DataFrame({'Mean_Blue': [means[0]], 'Mean_Green': [means[1]], 'Mean_Red': [means[2]], 'Standard_Blue': [stds[0]], 'Standard_Green': [stds[1]], 'Standard_Red': [stds[2]], 'Label': [make]})
    output = pd.concat([output, result], axis = 0)

    x_data = []
    for i in means:
        x_data.append(float(i))
    for i in stds:
        x_data.append(float(i))
    x_data = np.array(x_data)
    x_data = x_data.reshape(1,-1)
    pred = clf.predict(x_data)

    ########## cutted ##########
    # cutted_image = cutImage(imagePath)
    # (means, stds) = cv2.meanStdDev(cv2.cvtColor(cutted_image, cv2.COLOR_BGR2HSV))
    # cutted_result = pd.DataFrame({'Mean_Blue': [means[0]], 'Mean_Green': [means[1]], 'Mean_Red': [means[2]], 'Standard_Blue': [stds[0]], 'Standard_Green': [stds[1]], 'Standard_Red': [stds[2]], 'Label': [make]})
    # cutted_output = pd.concat([output, result], axis = 0)

    # cutted_x_data = []
    # for i in means:
        # cutted_x_data.append(float(i))
    # for i in stds:
        # cutted_x_data.append(float(i))
    # cutted_x_data = np.array(cutted_x_data)
    # cutted_x_data = x_data.reshape(1,-1)
    # cutted_pred = cutted_clf.predict(cutted_x_data)


    #original result	
    # print("original")
    # print("target: ", make)
    # print("predict: ", pred)

    if make == pred:
        correct = correct + 1

    #cutted result	
    # print("cutted")
    # print("target: ", make)
    # print("predict: ", cutted_pred)

    # if make == cutted_pred:
        # cutted_correct = cutted_correct + 1

print("Original")
print(correct/num_test_data)
# print("Cutted")
# print(cutted_correct/num_test_data)

tend = time.time()

print("time : ", total_data_num/(tend - tstart))

output.to_csv('Output.csv')
fig1 = plt.figure()
ax1 = Axes3D(fig1)
ax1.set_xlim3d(output['Mean_Blue'].min(), output['Mean_Blue'].max())
ax1.set_ylim3d(output['Mean_Green'].min(), output['Mean_Green'].max())
ax1.set_zlim3d(output['Mean_Red'].min(), output['Mean_Red'].max())
for i in range(output.shape[0]):
    if output.iloc[i]['Label'] == 'white':
        ax1.plot(output.iloc[i]['Mean_Blue'], output.iloc[i]['Mean_Green'], output.iloc[i]['Mean_Red'], 'ro')
    elif output.iloc[i]['Label'] == 'steak':
        ax1.plot(output.iloc[i]['Mean_Blue'], output.iloc[i]['Mean_Green'], output.iloc[i]['Mean_Red'], 'go')
    elif output.iloc[i]['Label'] == 'curl':
        ax1.plot(output.iloc[i]['Mean_Blue'], output.iloc[i]['Mean_Green'], output.iloc[i]['Mean_Red'], 'bo')
    elif output.iloc[i]['Label'] == 'dressed_long_leg':
        ax1.plot(output.iloc[i]['Mean_Blue'], output.iloc[i]['Mean_Green'], output.iloc[i]['Mean_Red'], 'co')
    elif output.iloc[i]['Label'] == 'long leg':
        ax1.plot(output.iloc[i]['Mean_Blue'], output.iloc[i]['Mean_Green'], output.iloc[i]['Mean_Red'], 'mo')
    else:
        ax1.plot(output.iloc[i]['Mean_Blue'], output.iloc[i]['Mean_Green'], output.iloc[i]['Mean_Red'], 'ko')
plt.show()

fig2 = plt.figure()
ax2 = Axes3D(fig2)
ax2.set_xlim3d(output['Standard_Blue'].min(), output['Standard_Blue'].max())
ax2.set_ylim3d(output['Standard_Green'].min(), output['Standard_Green'].max())
ax2.set_zlim3d(output['Standard_Red'].min(), output['Standard_Red'].max())
for i in range(output.shape[0]):
    if output.iloc[i]['Label'] == 'white':
        ax2.plot(output.iloc[i]['Standard_Blue'], output.iloc[i]['Standard_Green'], output.iloc[i]['Standard_Red'], 'ro')
    elif output.iloc[i]['Label'] == 'steak':
        ax2.plot(output.iloc[i]['Standard_Blue'], output.iloc[i]['Standard_Green'], output.iloc[i]['Standard_Red'], 'go')
    elif output.iloc[i]['Label'] == 'curl':
        ax2.plot(output.iloc[i]['Standard_Blue'], output.iloc[i]['Standard_Green'], output.iloc[i]['Standard_Red'], 'bo')
    elif output.iloc[i]['Label'] == 'dressed_long_leg':
        ax2.plot(output.iloc[i]['Standard_Blue'], output.iloc[i]['Standard_Green'], output.iloc[i]['Standard_Red'], 'co')
    elif output.iloc[i]['Label'] == 'long leg':
        ax2.plot(output.iloc[i]['Standard_Blue'], output.iloc[i]['Standard_Green'], output.iloc[i]['Standard_Red'], 'mo')
    else:
        ax2.plot(output.iloc[i]['Standard_Blue'], output.iloc[i]['Standard_Green'], output.iloc[i]['Standard_Red'], 'ko')
plt.show()

# cutted_output.to_csv('Cutted_Output.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'train'

# 4.gaussian

In [ ]:
#載入必要模組
from __future__ import print_function
from sklearn.svm import SVC
from skimage import exposure
from skimage import feature
from imutils import paths
import argparse
import imutils
import cv2
from sklearn import tree
import numpy as np
import os
import glob
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
import random
import statistics
import time
#data用來存放HOG資訊，labels則是存放對應的標籤
labels = []
count = int(0)
data = np.empty((0, 6),float)
total_data_num = 0
#依序讀取training dataset中的圖檔
entries = os.listdir("train")
prediction = []
allimage_list = []

for dirPath in entries:
    allimage = glob.glob( "train/"+dirPath +"/*")
    label = 0
    for i in allimage:
        allimage_list.append(i)
total_data_num = len(allimage_list)
temp_data_num = total_data_num

while temp_data_num > total_data_num * 0.3:

        imagePath = allimage_list.pop(random.randint(0, temp_data_num - 1))
        temp_data_num = temp_data_num - 1
        #將資料夾的名稱取出作為該圖檔的標籤
        make = imagePath.split("/")[-1]
        make = make.split("\\")[-2]
        if make == 'curl':
            label = 1
        elif make == 'dressed_long_leg':
            label = 2
        elif make == 'long leg':
            label = 3
        elif make == 'steak':
            label =4
        elif make == 'white':
            label =5
        elif make == 'small white':
            label = 6

        #----以下為訓練圖檔的預處理----# 

        #載入圖檔,轉為灰階,作模糊化處理
        image = cv2.imread(imagePath)
        (means, stds) = cv2.meanStdDev(cv2.cvtColor(image, cv2.COLOR_BGR2HSV))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #將裁切後的圖檔尺寸更改為500x500。
        Cutted = cv2.resize(gray, (500, 500))
        x_data = []
        for i in means:
            x_data.append(float(i))
        for i in stds:
            x_data.append(float(i))
        x_data = np.array(x_data)

        data = np.vstack((data, x_data))
        labels.append(label)
        count = count + 1
        if count % 10 == 0:
            print (count, end="\r")
data = np.array(data)
Gaussian = GaussianProcessClassifier(1.0 * RBF(1.0))
Gaussian.fit(data, labels)
fps_list = []
correct = 0
num_test_data = 0
while temp_data_num:
    tstart = time.time()
    temp_data_num = temp_data_num - 1
    imagePath = allimage_list.pop()
    make = imagePath.split("/")[-1]
    make = make.split("\\")[-2]
    # 從測試資料中讀取圖檔
    image = cv2.imread(imagePath)
    # ----以下為測試圖檔的預處理----# 
    # 轉為灰階並模糊化
    (means, stds) = cv2.meanStdDev(cv2.cvtColor(image, cv2.COLOR_BGR2HSV))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # 將裁切後的圖檔尺寸更改為500x500。
    Cutted = cv2.resize(gray, (500, 500))

    x_data = []
    for i in means:
        x_data.append(float(i))
    for i in stds:
        x_data.append(float(i))
    # for i in H:
        # x_data.append(float(i))
    x_data = np.array(x_data)
    x_data = x_data.reshape(1,-1)
    pred = Gaussian.predict(x_data)
    if pred < 1.5 and pred >= 0.5:
        prediction = 'curl'
    elif pred < 2.5 and pred >= 1.5:
        prediction = 'dressed_long_leg'
    elif pred < 3.5 and pred >= 2.5:
        prediction = 'long leg'
    elif pred < 4.5 and pred >= 3.5:
        prediction = 'steak'
    elif pred < 5.5 and pred >= 4.5:
        prediction = 'white'
    elif  pred < 6.5 and pred >= 5.5:
        prediction = 'small white'

    num_test_data = num_test_data + 1
    if make == prediction:
        correct = correct + 1
    tend = time.time()
    FPS_all = 1/(tend-tstart)
    fps_list.append(FPS_all)
mean = statistics.mean(fps_list)
print("Average of FPS: ", mean)
print("Accuracy: ", (correct/num_test_data)*100, "%")

FileNotFoundError: [Errno 2] No such file or directory: 'train'